### <mark>**✅Crawling_Coupang**

##### **Library**

In [1]:
# selenium import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

# library import
from bs4 import BeautifulSoup
from dateutil import parser
import time
import pandas as pd
import platform
import os
from time import sleep
from glob import glob
from tqdm import tqdm
from datetime import datetime
import pyautogui as pg

##### **Accept**

In [4]:
os_name = platform.system().lower()
architecture = platform.machine()

driver_path = None
if os_name == 'darwin':
    driver_path = glob('../driver/**/chromedriver', recursive=True)[0]
else:
    driver_path = glob('../driver/**/chromedriver.exe', recursive=True)[0]

In [5]:
# 드라이버 파일의 권한을 확인하고 수정
if os_name == 'darwin':  # 맥 사용자
    driver_permissions = os.stat(driver_path).st_mode
    if not (driver_permissions & 0o100):  # 실행 권한 확인
        os.chmod(driver_path, driver_permissions | 0o111)  # 실행 권한 추가
else:  # 윈도우 사용자
    driver_permissions = os.stat(driver_path).st_mode
    if not (driver_permissions & 0o100):  # 실행 권한 확인
        os.chmod(driver_path, driver_permissions | 0o111)  # 실행 권한 추가

##### **Chrome Options**

In [6]:
service = Service(executable_path=driver_path)
chrome_options =  webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

##### **Define**

In [7]:
url = 'https://www.coupang.com/vp/products/8595318830?itemId=24933894925&vendorItemId=91940018148'

In [8]:
# Web OPEN
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url)
time.sleep(3)

In [9]:
# Web ZOOM
driver.execute_script("document.body.style.zoom='10%'")
# Web Height
last_height = driver.execute_script("return document.body.scrollHeight")

# Page Loading

while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(1)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [10]:
# Parsing
soup = BeautifulSoup(driver.page_source, 'html.parser')
data = []

In [12]:
# Selector
review = '#btfTab > ul.tab-contents > li.product-review.tab-contents__content > div > div.sdp-review__article.js_reviewArticleContainer > section.js_reviewArticleListContainer'
id = '#btfTab > ul.tab-contents > li.product-review.tab-contents__content > div > div.sdp-review__article.js_reviewArticleContainer > section.js_reviewArticleListContainer > article:nth-child(3) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__user > span'
created = '#btfTab > ul.tab-contents > li.product-review.tab-contents__content > div > div.sdp-review__article.js_reviewArticleContainer > section.js_reviewArticleListContainer > article:nth-child(3) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__reg-date'
content = '#btfTab > ul.tab-contents > li.product-review.tab-contents__content > div > div.sdp-review__article.js_reviewArticleContainer > section.js_reviewArticleListContainer > article:nth-child(3) > div.sdp-review__article__list__review.js_reviewArticleContentContainer > div'

count = (soup.select_one('#btfTab > ul.tab-contents > li.product-review.tab-contents__content > div > div.sdp-review__average.js_reviewAverageContainer > section.sdp-review__average__total-star > div.sdp-review__average__total-star__info > div.sdp-review__average__total-star__info-count').text)
selector_title = soup.select_one('#contents > div.prod-atf > div.prod-atf-main > div.prod-buy.new-oos-style.not-loyalty-member.eligible-address.without-subscribe-buy-type.DISPLAY_0.only-one-delivery.fix-verdor-section-display.move-atc-and-buy-now-ctas-atf.prod-buy-alias.update-price-section-style-with-rds > div.prod-buy-header > h1')

In [15]:
# Count page
page_int = int(count.replace(',', ''))

def calculate_page_no(page_int):
    page_no = page_int // 5 if page_int % 5 == 0 else page_int // 5 + 1
    return page_no

page_no = calculate_page_no(page_int)

##### **Crawling**

In [16]:
for page in tqdm(range(1, page_no + 1), desc="현재 페이지", unit="page", leave=False):
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, review))
    )
    
    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    empty_review_count = 0

    review_articles = soup.select(f"{review} article")

    for article in review_articles:
        try:
            id_selector = article.select_one("div.sdp-review__article__list__info > div.sdp-review__article__list__info__user > span")
            user_id = id_selector.get_text(strip=True) if id_selector else "N/A"

            created_selector = article.select_one("div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__reg-date")
            created_at = parser.parse(created_selector.get_text(strip=True)) if created_selector else "N/A"

            content_selector = article.select_one("div.sdp-review__article__list__review.js_reviewArticleContentContainer > div")
            review_text = content_selector.get_text(strip=True) if content_selector else None

            if review_text is None:
                empty_review_count += 1
                continue

            data.append({
                "ID": user_id,
                "CREATED": created_at,
                "REVIEW": review_text
            })
        except Exception as e:
            print(f"Error processing review: {e}")

    # 빈 리뷰가 페이지 전체 리뷰 개수와 같다면 크롤링 종료
    if empty_review_count == len(review_articles):
        break

    # 다음 페이지 버튼 클릭
    if page < page_no:
        try:
            next_page_element = driver.find_element(
                By.CSS_SELECTOR,
                f'#btfTab > ul.tab-contents > li.product-review.tab-contents__content > div > div.sdp-review__article.js_reviewArticleContainer > section.js_reviewArticleListContainer > div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child({page + 2})'
            )
            driver.execute_script("arguments[0].click();", next_page_element)
        except Exception as e:
            print(f"Error clicking next page: {e}")
            break


현재 페이지:   0%|          | 0/51 [00:00<?, ?page/s]

UnexpectedAlertPresentException: Alert Text: 서버에서 오류가 발생하였습니다.
Message: unexpected alert open: {Alert text : 서버에서 오류가 발생하였습니다.}
  (Session info: chrome=134.0.6998.35)
Stacktrace:
	GetHandleVerifier [0x00007FF62EA5C6A5+28789]
	(No symbol) [0x00007FF62E9C5B20]
	(No symbol) [0x00007FF62E858F9A]
	(No symbol) [0x00007FF62E90042B]
	(No symbol) [0x00007FF62E8D7103]
	(No symbol) [0x00007FF62E89FFC0]
	(No symbol) [0x00007FF62E8A1273]
	GetHandleVerifier [0x00007FF62EDA1AED+3458237]
	GetHandleVerifier [0x00007FF62EDB829C+3550316]
	GetHandleVerifier [0x00007FF62EDADB9D+3507565]
	GetHandleVerifier [0x00007FF62EB22C6A+841274]
	(No symbol) [0x00007FF62E9D09EF]
	(No symbol) [0x00007FF62E9CCB34]
	(No symbol) [0x00007FF62E9CCCD6]
	(No symbol) [0x00007FF62E9BC119]
	BaseThreadInitThunk [0x00007FFD44ADE8D7+23]
	RtlUserThreadStart [0x00007FFD465FBF6C+44]


In [36]:
for page in tqdm(range(1, page_no), desc="현재 페이지", unit="page", leave=False):
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, review))
    )

    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    empty_review_count = 0

    for selector in soup.select(review):
        id_selector = selector.select_one(id).text
        created_selector = parser.parse(selector.select_one(created).text)
        content_selector = selector.select_one(content)
        if content_selector is None:
            empty_review_count += 1
            continue

        data.append({
            'ID': id_selector,
            'CREATED': created_selector,
            'REVIEW': content_selector
        })

    if empty_review_count == len(soup.select(review)):
        break

    if page < (page_no + 1):
        next_page_element = driver.find_element(By.CSS_SELECTOR, f'#btfTab > ul.tab-contents > li.product-review.tab-contents__content > div > div.sdp-review__article.js_reviewArticleContainer > section.js_reviewArticleListContainer > div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child({page + 2})')
        driver.execute_script("arguments[0].click();", next_page_element)

##### **To CSV**

In [38]:
result_df = pd.DataFrame(data)
result_df['CREATED'] = result_df['CREATED'].apply(lambda x: f"{x.day:02d}-{x.month:02d}-{str(x.year)[2:]}")

In [39]:
driver.close()
driver.quit()

In [40]:
split_text = selector_title.text.split(" ")
if len(split_text) >= 3:
    file_name = " ".join(split_text[:2])
else:
    file_name = selector_title.text

AttributeError: 'NoneType' object has no attribute 'text'

In [41]:
today_str = datetime.today().strftime("%m%d")

In [45]:
result_df.to_csv(rf'..\data\review\{today_str}_.csv', index=False, encoding='utf-8-sig')